# Project 3 - Sequence models

igu011 and edj001

In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from datetime import datetime
from torch.utils.data import DataLoader, TensorDataset
from rich import print

import numpy as np
import torchtext
from os import listdir
import re
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

torch.manual_seed(265)
torch.set_default_dtype(torch.double)

device = torch.device("cpu")
print(f"Device {device}.")


/home/elias/miniconda3/envs/265_2/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device cpu.

## 2.1 Word embedding

#### 1 - Read txt files and tokenize them to obtain train/validation/test lists of words.


In [2]:
# tokenizer will split a long text into a list of english words
tokenizer = get_tokenizer("basic_english")


def read_files(datapath="./"):
    """
    Return a list of strings, one for each line in each .txt files in 'datapath'
    """
    # Find all txt files in directory
    files = listdir(datapath)
    files = [datapath + f for f in files if f.endswith(".txt")]

    # Stores each line of each book in a list
    lines = []
    for f_name in files:
        with open(f_name) as f:
            lines += f.readlines()
    return lines


# Match any word containing digit
no_digits = "\w*[0-9]+\w*"
# Match word containing a uppercase
no_names = "\w*[A-Z]+\w*"
# Match any sequence containing more than one space
no_spaces = "\s+"


def tokenize(lines):
    """
    Tokenize the list of lines
    """
    list_text = []
    for line in lines:
        list_text += tokenizer(line)
    return list_text


def yield_tokens(lines):
    """
    Yield tokens, ignoring names and digits to build vocabulary
    """
    for line in lines:
        line = re.sub(no_digits + "|" + no_names, " ", line)
        line = re.sub(no_spaces, " ", line)
        yield tokenizer(line)


def count_freqs(data, vocab):
    """
    Count occurrences of each word in vocabulary in the data
    """
    freqs = torch.zeros(len(vocab), dtype=torch.int)
    for w in data:
        freqs[vocab[w]] += 1
    return freqs


train_books = read_files(datapath="./data_train/")
train_tokenized = tokenize(train_books)

val_books = read_files(datapath="./data_val/")
val_tokenized = tokenize(val_books)

test_books = read_files(datapath="./data_test/")
test_tokenized = tokenize(test_books)


#### 2 - Define a vocabulary based on the training dataset
To avoid getting a too large vocabulary, a solution can be to keep only words that appear at least 100 times in the training dataset.
Report the total number of words in the training dataset, the number of distinct words in the
training dataset and the size the defined vocabulary. Comment your results.

In [3]:
# vocab contains the vocabulary found in the data, associating an index to each word
specials = ["<unk>", ",", ".", "!", "?", "-", "&"]
vocab = build_vocab_from_iterator(yield_tokens(train_books), min_freq=100, specials=specials)
# Since we removed all words with an uppercase when building the vocabulary, we skipped the word "I"
vocab.append_token("i")

# Value of default index. This index will be returned when OOV (Out Of Vocabulary) token is queried.
vocab.set_default_index(vocab["<unk>"])
vocab_size = len(vocab)

print("Total number of words in the dataset:   ", len(train_tokenized))
print("Number of distinct words in the dataset:", len(set(train_tokenized)))
print("Size the defined vocabular:             ", vocab_size)


freqs = count_freqs(train_tokenized, vocab)
top20 = freqs[len(specials) : len(specials) + 19]
print(
    "Top 20 occurences:\n",
    [
        (f.item(), w)
        for (f, w) in zip(top20, vocab.lookup_tokens(range(len(specials), len(specials) + 19)))
    ],
)


Total number of words in the dataset:    1368807

Number of distinct words in the dataset: 30374

Size the defined vocabular:              1053

occurences:

[
    (251033, '<unk>'),
    (89904, ','),
    (63121, '.'),
    (7122, '!'),
    (6510, '?'),
    (7, '-'),
    (0, '_'),
    (0, "'"),
    (22, '&'),
    (71106, 'the'),
    (43426, 'and'),
    (33952, 'to'),
    (30061, 'of'),
    (23575, 'a'),
    (18657, 'in'),
    (20755, 'he'),
    (16814, 'that'),
    (15056, 'was'),
    (14400, 'his')
]

In [4]:
def create_dataset(text, vocab, context_size=3):
    contexts = []
    targets = []
    n_text = len(text)

    # Transform the text as a list of integers.
    txt = [vocab[w] for w in text]

    for i in range(n_text - context_size):

        t = txt[i + context_size]

        c = torch.Tensor(txt[i : i + context_size]).type(torch.long)

        targets.append(t)

        contexts.append(c)

    contexts = torch.stack(contexts)
    targets = torch.tensor(targets)
    return TensorDataset(contexts, targets)


data = create_dataset(train_tokenized, vocab)


#### 3 - Define a n-gram language model architecture based on this vocabulary that contains an embedding layer.
To drastically reduce computational cost, the dimension of the embedding can be as low as 16 even though in a real setting a larger space would be used.

In [5]:
class NGram(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGram, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((inputs.shape[0], -1))
        out = self.linear1(embeds)
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


#### 4 - Train several models, select the best one and evaluate its performance.
Note that the accuracy here is potentially extremely low, but the real objective is not to train a good predictor, only
to have a good representation of the semantic of each word in the vocabulary

In [6]:
def train(n_epochs, optimizer, model, loss_fn, train_loader):
    """
    Train our model and save weight values
    """
    n_batch = len(train_loader)
    losses_train = []
    model.train()
    optimizer.zero_grad(set_to_none=True)

    for epoch in range(1, n_epochs + 1):

        loss_train = 0.0
        for contexts, labels in train_loader:

            contexts = contexts.to(device=device)
            labels = labels.to(device=device)

            outputs = model(contexts)

            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            loss_train += loss.item()

        losses_train.append(loss_train / n_batch)

        print(
            "{}  |  Epoch {}  |  Training loss {:.5f}".format(
                datetime.now().time(), epoch, loss_train / n_batch
            )
        )
    return


def relative_error(a, b):
    return torch.norm(a - b) / torch.norm(a)


In [7]:
train_loader = DataLoader(data, batch_size=64, shuffle=False)
loss_fn = nn.NLLLoss()

n_epochs = 4
lr = 0.1

NGram_model = NGram(vocab_size, 16, 3).to(device=device)
optimizer = optim.SGD(NGram_model.parameters(), lr=lr)

weight = train(
    n_epochs=n_epochs,
    optimizer=optimizer,
    model=NGram_model,
    loss_fn=loss_fn,
    train_loader=train_loader,
)


18:19:27.976049  |  Epoch 1  |  Training loss 4.16732

18:20:29.579370  |  Epoch 2  |  Training loss 3.96308

18:21:35.413986  |  Epoch 3  |  Training loss 3.89900

18:22:40.719907  |  Epoch 4  |  Training loss 3.86028

In [8]:
n_epochs = 15
lr = 0.1

NGram_larger_model = NGram(vocab_size, 64, 3).to(device=device)
NGram_larger_optimizer = optim.SGD(NGram_larger_model.parameters(), lr=lr)

NGram_larger_model_weight = train(
    n_epochs=n_epochs,
    optimizer=NGram_larger_optimizer,
    model=NGram_larger_model,
    loss_fn=loss_fn,
    train_loader=train_loader,
)


18:23:50.942429  |  Epoch 1  |  Training loss 4.01377

18:24:54.074422  |  Epoch 2  |  Training loss 3.83470

18:25:56.106352  |  Epoch 3  |  Training loss 3.77931

18:27:00.906138  |  Epoch 4  |  Training loss 3.74572

18:28:05.167979  |  Epoch 5  |  Training loss 3.72192

...

18:34:33.401388  |  Epoch 11  |  Training loss 3.64971

18:35:34.077173  |  Epoch 12  |  Training loss 3.64268

18:36:26.086427  |  Epoch 13  |  Training loss 3.63636

18:37:14.582582  |  Epoch 14  |  Training loss 3.63062

18:38:01.914395  |  Epoch 15  |  Training loss 3.62538

In [18]:
def accuracy(model, val_loader):
    """
    Compute the accuracy of our model on the validation set
    """
    model.eval()
    correct = 0
    with torch.no_grad():
        for contexts, labels in val_loader:
            contexts = contexts.to(device=device)
            labels = labels.to(device=device)

            outputs = model(contexts)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

    return correct / len(val_loader.dataset)

TODO: SELECT BEST MODEL BASED ON VAL AND CHECH TEST PERFORMANCE.

In [24]:
val_data = create_dataset(val_tokenized, vocab)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False) # Maybe create loaders at the top

train_acc = accuracy(NGram_larger_model, train_loader)
val_acc = accuracy(NGram_larger_model, val_loader)

print(f"Training accuracy: {train_acc:.1%}")
print(f"Validation accuracy: {val_acc:.1%}")

Training accuracy: 21.8%

Validation accuracy: 22.9%

#### 5 - Compute the cosine similarity matrix of the vocabulary based on the trained embedding.
For some words of your choice (e.g. me, white, man, have, be, child, yes, what etc.), report the 10
most similar words. Comment your results.

In [9]:
wheights = NGram_larger_model.embeddings.weight.detach()

sim_matrix = np.zeros(shape=(len(wheights), len(wheights)))

for w1 in range(0, len(wheights)):
    for w2 in range(0, len(wheights)):
        # Cosine similarity
        sim_matrix[w1][w2] = np.dot(wheights[w1], wheights[w2]) / (
            np.linalg.norm(wheights[w1]) * np.linalg.norm(wheights[w2])
        )


In [16]:
testing_words = ["me", "white", "man", "have", "be", "child", "yes", "what"]
testing_words_idx = [(i, vocab.get_stoi()[i]) for i in testing_words]

n_most_similar = 10
for w in testing_words_idx:

    indices = (-sim_matrix[w[1]]).argsort()[:n_most_similar]
    sim_words = [(vocab.get_itos()[i], sim_matrix[w[1]][i]) for i in indices]
    print(f"{w} most similar words:")

    for sim_w in sim_words:
        print(f"\t{sim_w}")


('me', 40) most similar words:

('me', 1.0)

('lifted', 0.37033130282712134)

('indeed', 0.320596876343763)

('activity', 0.31892116863226155)

...

('start', 0.3448425148437706)

('whom', 0.33315547531447476)

('various', 0.329404409269461)

('stand', 0.32341906141438814)

('everything', 0.31233032787460996)

TODO: COMMENT HERE

#### 6 -Visualize the embedding space.
To do so, upload the vocabulary and their corresponding values in the embedding space as tsv files. Try to find
and select clusters. Report both plots and their corresponding selections for some meaningful
clusters.

In [ ]:
import csv

embedding_wheights = NGram_larger_model.embeddings.weight.detach()

with open("./wheights.tsv", "wt") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for i in embedding_wheights:
        tsv_writer.writerow(i.numpy())


with open("./metadata.tsv", "wt") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    vocab_dict = vocab.get_itos()
    for i in vocab_dict:
        tsv_writer.writerow([i])


TODO: PUT VISUALIZATIONS HERE

## 2.2 Conjugating *be* and *have*

#### 1 - Use your trained word embedding and define a MLP architecture as well as a RNN architecture to predict be and have conjugation given the context around the missing target. 
Use the same context size for both MLPs and RNNs, even though RNNs could take a context size of arbitrary
length.

#### 2 - Train several models, select the best one and evaluate its performance. 
Comment the differences in terms of performances/training time between the MLP architecture and the RNN
architecture.

## 2.3 Text generation

#### 1 -  Use your trained word embedding and define a RNN architecture that can predict the next word given the context before the target.


#### 2 - Train several models, select the best one and evaluate its performance.
Note that the accuracy here is potentially very low.


#### 3 - Implement the beam search algorithm.

#### 4 - Have fun playing with your model. 
Start a sentence, give it to your model, and let it complete the next n words using the beam search algorithm. Report and comment your results.